# Contrastive Language-Image Pretraining with SogCLR

### **Introduction**

In this tutorial, you will learn how to conduct contrastive language-image pretraining by optimizing the [Global Contrastive Loss](https://arxiv.org/abs/2202.12387) (GCL) on a subset of the [Conceptual Captions](https://ai.google.com/research/ConceptualCaptions/) dataset. Also, you will learn how to evaluate the model on retrieval task using the [MSCOCO](https://cocodataset.org/#home) dataset and zero-shot classification task using the [ImageNet](https://www.image-net.org/challenges/LSVRC/index.php) dataset. The code is based on [iSogCLR's](https://github.com/zhqiu/contrastive-learning-iSogCLR) codebase, which includes the implementation of CLIP, SogCLR and iSogCLR.

### Preparation

First, we:

1. Download the source code and data
2. Install required packages

### Training

The following command runs the training script to train a ResNet50 (pretrained on ImageNet) and a DistilBERT (pretrained on BookCorpus and English Wikipedia) on the cc3m dataset using the SogCLR loss for 30 epochs with temperature 0.01.

In [30]:
!python ../main.py \
    --data_path ../../datasets  \
    --ann_path ../../datasets/clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir ../../output/sogclr_cc3m_g0.8_e30 \
    --init_model \
    --use_amp \
    --ita_type sogclr \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --device cuda \
    --val_frequency 5 \
    --step_size_per_epoch 200 \
    --print_freq_per_epoch 100 \
    --epochs 1

***
Creating retrieval dataset
***
len of train_dataset: 100000
len of validation dataset: 5000
***
Creating model
***
Cosine annealing scheduler will have no effect on the learning rate since t_initial = t_mul = eta_mul = 1.
Start training
Train Epoch: [0]: 100%|█| 781/781 [08:52<00:00,  1.47it/s, loss_ita=0.0104, lr=0
Averaged stats: lr: 0.0001  lr_temp_net: 0.0000  loss_ita: 0.0228  avg_image_tau: 0.0000  avg_text_tau: 0.0000  cur_eta: 0.0000  grad_tau_image: 0.0000  grad_tau_text: 0.0000  b_I: 0.0000  b_T: 0.0000  v: 0.0000  lamda: 0.0000  weights_image_pos: 0.0000  weights_text_pos: 0.0000
Training time: 0:08:57


### Evaluation

The following command runs the evaluation script to evaluate the retrieval performance of the trained model on the MSCOCO validation dataset and the zero-shot classification performance on the ImageNet validation dataset. The evaluation command is obtained by appending `--evaluate --checkpoint /path/to/your/checkpoint --zs_dataset imagenet --zs_datafolder /path/to/imagenet/val` to the training command.

In [19]:
!python ../main.py \
    --data_path ../../datasets \
    --ann_path ../../datasets/clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --checkpoint '../final_model/checkpoint_20.pth' \
    --zs_dataset imagenet \
    --zs_datafolder ../../datasets/imagenet/val \
    --evaluate True \
    --device cpu


Args before running the pipeline: 
Namespace(data_path='../../datasets', ann_path='../../datasets/clip_train', train_file='../../datasets/clip_train\\cc3m_train_subset.json', train_image_root='../../datasets\\cc3m_subset_100k', bert_config='configs/config_bert.json', image_encoder='resnet50', text_encoder='distilbert-base-uncased', image_res=256, embed_dim=256, optimizer='adamW', sched='cosine', lr=0.0002, wd_temp_net=0.001, min_lr=1e-06, warmup=True, warmup_lr=1e-05, weight_decay=0.02, decay_rate=1, epochs=30, warmup_epochs=5, cooldown_epochs=0, patience_epochs=5, decay_epochs=3, momentum=0, init_model=True, batch_size_train=128, batch_size_test=128, evaluate=True, val_frequency=2, checkpoint="'../final_model/checkpoint_20.pth'", device='cpu', seed=42, world_size=1, dist_url='env://', distributed=False, no_distributed=False, step_size_per_epoch=100, print_freq_per_epoch=100, resume_learning=False, k_test=256, output_dir='./output/clip_test', loss_function='isogclr_new', vicreg_sim_coe

### Benchmarks

The following results are recall at 1 results on the provided MSCOCO and ImageNet datasets. The first row of results are from the model trained using the CLIP loss, and the second row of results are from the model trained using the SogCLR loss. All results are based on a batch size of 128 for 30-epoch pretraining. IR@1 denotes the recall at 1 of image retrieval on MSCOCO, TR@1 denotes the recall at 1 of text retrieval on MSCOCO, and ACC@1 denotes the top 1 accuracy on ImageNet. Average denotes the average of the three metrics.

| Method | MSCOCO TR@1 | MSCOCO IR@1 | ImageNet ACC@1 | Average |
|:----------:|:--------:|:--------:|:--------:|:--------:|
| CLIP | 12.0 | 9.32 | 21.35 | 14.22 |
| SogCLR |  14.38  |  10.73  | 24.54 | 16.55 |